# Embedding Similarity & Weight Projection - M2V

After extracting the learned node embeddings from the LastFM database using Metapath2Vec, we will input and process the respective CSV and txt files to calculate `Cosine Similarity` between any two nodes sharing an edge in the original graph.

We first import the required libraries.

In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

## Loading Embeddings Data from CSV

Since your embeddings are saved in a CSV file, we will use Pandas to load this file into a DataFrame. Each row in CSV file represents a node, and each column represents a feature of the embeddings (i.e., 128-dimension embeddings).

In [6]:
embeddings_df = pd.read_csv('M2V_Embeddings/node_embeddings.csv', delimiter=',', header=None, float_precision='high')

with open('M2V_Embeddings/node_ids.txt', 'r') as file:
    node_indexes = [line.strip() for line in file]

# Add node_indexes back as the first column of the DataFrame
embeddings_df.insert(0, 'node_id', node_indexes)

# Set node indexes as embeddings_df index to allow for faster search later on
embeddings_df.set_index('node_id', inplace=True)

# Now 'embeddings_df' is ready for further analysis
print(embeddings_df.head())

embeddings_df.shape

              0         1         2         3         4         5         6    \
node_id                                                                         
t_73    -0.193153  0.081209  0.221022 -0.237067 -0.341212 -0.124392  0.518349   
t_24    -0.378580  0.373095  0.419321 -0.144066 -0.177232 -0.074142  0.437567   
t_79    -0.360389  0.083494  0.088490 -0.351976 -0.175297 -0.167279  0.257180   
t_18    -0.356609  0.005678  0.402312 -0.554622 -0.022824  0.020642 -0.036826   
t_81    -0.248152 -0.063090  0.202308 -0.197242 -0.130315 -0.201546  0.333233   

              7         8         9    ...       118       119       120  \
node_id                                ...                                 
t_73     0.239695 -0.272634 -0.135897  ...  0.174696  0.110674  0.100949   
t_24     0.540218 -0.267260  0.076700  ...  0.452497  0.030775  0.230328   
t_79     0.460405 -0.298290 -0.080821  ...  0.350925  0.013836  0.011780   
t_18     0.425137 -0.540539 -0.208693  ...  0.457652

(21518, 128)

Now that we have cleaned-up the embeddings into a dataframe, we need to check if there are any inconsistencies in the data. We also check for non-numeric data.

In [7]:
# Check for non-numeric data
print("Data types:\n", embeddings_df.dtypes)

# Check for missing values
if embeddings_df.isnull().values.any():
    print("Missing values found")

# Check shape of embeddings dataframe to see if there are varying row lengths
print("DataFrame shape:", embeddings_df.shape)


Data types:
 0      float64
1      float64
2      float64
3      float64
4      float64
        ...   
123    float64
124    float64
125    float64
126    float64
127    float64
Length: 128, dtype: object
DataFrame shape: (21518, 128)


## Loading Edge List Data from .edgelist File

To be able to access which nodes are connected by an edge, we need to import the edge list into another dataframe. Note that the node IDs must be consistent across both the embedding and edge list dataframes! It is also an undirected graph, meaning source and target do not necessarily mean anything.

In [8]:
# File path
edgelist_file = 'EdgeList_LastFM/lastfm.edgelist'

# Read edge list into DataFrame
edge_list_df = pd.read_csv(edgelist_file, sep=' ', header=None, names=['source', 'target'])

display(edge_list_df.head())

display(embeddings_df.head())

,source,target
0,u_2,a_51
1,u_2,a_52
2,u_2,a_53
3,u_2,a_54
4,u_2,a_55


,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
node_id,,,,,,,,,,,,,,,,,,,,,
t_73,-0.193153,0.081209,0.221022,-0.237067,-0.341212,-0.124392,0.518349,0.239695,-0.272634,-0.135897,...,0.174696,0.110674,0.100949,0.033665,0.075441,0.331266,0.020202,-0.213357,-0.107091,0.157276
t_24,-0.378580,0.373095,0.419321,-0.144066,-0.177232,-0.074142,0.437567,0.540218,-0.267260,0.076700,...,0.452497,0.030775,0.230328,-0.338027,0.048291,0.491815,0.106747,-0.172917,-0.281297,0.118938
t_79,-0.360389,0.083494,0.088490,-0.351976,-0.175297,-0.167279,0.257180,0.460405,-0.298290,-0.080821,...,0.350925,0.013836,0.011780,-0.139148,-0.189243,0.288592,-0.111147,0.163391,-0.107350,0.362807
t_18,-0.356609,0.005678,0.402312,-0.554622,-0.022824,0.020642,-0.036826,0.425137,-0.540539,-0.208693,...,0.457652,0.355199,-0.175280,-0.539665,-0.307221,0.262405,0.137945,-0.038897,-0.614187,0.010464
t_81,-0.248152,-0.063090,0.202308,-0.197242,-0.130315,-0.201546,0.333233,0.321245,-0.580546,0.083575,...,0.029460,0.599169,0.125027,-0.064965,-0.415897,0.071050,-0.046679,0.193068,-0.339528,0.056438


## Calculating Cosine Similarity

- For each edge, we retrieve the embeddings of the connected nodes.
- Use cosine_similarity from sklearn.metrics.pairwise to calculate the similarity for each edge.
- Store the similarity values in a new column in the edge list DataFrame.

### Method 1: Row-by-Row Iteration (Slower, Inefficient)

For graphs with a very large number of edges, iterating over each row using DataFrame.iterrows() and calculating cosine similarity one pair at a time can be very inefficient. This method has a time complexity that grows linearly with the number of edges, leading to long execution times for large graphs. 

In [5]:
# Assume embeddings_df is your DataFrame with embeddings indexed by node IDs
# Calculate cosine similarities
similarities = []
for _, row in edge_list_df.iterrows():
    emb1 = embeddings_df.loc[row['source']].values.reshape(1, -1)
    emb2 = embeddings_df.loc[row['target']].values.reshape(1, -1)
    similarity = cosine_similarity(emb1, emb2)[0, 0]
    similarities.append(similarity)

# Add similarities to the edge list DataFrame
edge_list_df['weight'] = similarities

KeyboardInterrupt: 

### Method 2: Batch Processing using Vectorization (Faster, Efficient)

1. Efficiency and Vectorization
    - Vectorized Operations: Modern CPUs and computing frameworks like NumPy are optimized for vectorized operations, where the same operation is performed simultaneously on multiple data points. This is inherently more efficient than processing each data point (or in this case, each pair of embeddings) individually, as it minimizes the overhead associated with looping constructs in high-level languages like Python.

    - Batch Processing: By processing multiple pairs of embeddings at once, the batch approach reduces the number of iterations and takes full advantage of vectorized operations. This leads to a significant reduction in computation time, especially for large datasets.

2. Scalability
    - Memory Management: Calculating cosine similarities for millions of edges at once can be memory-intensive, leading to memory overflow or significantly slowed performance due to swapping. Processing the data in smaller batches helps manage memory usage more effectively, ensuring that the computation remains within the available system resources, thereby maintaining performance across varying scales of data.

    - Parallelization Potential: Although not implemented in the provided code, batch processing opens up possibilities for parallel computation. Batches can be processed in parallel across multiple CPU cores or even distributed systems, further speeding up the computation for very large graphs.

3. Practicality
    - Adaptability: The batch size can be adjusted based on the available computing resources and the specific requirements of the dataset. This flexibility allows the method to be optimized for different environments, from personal laptops to high-performance computing clusters.

    - Reduced Computational Overhead: The original method's reliance on DataFrame.iterrows() is known to be inefficient for large datasets due to the overhead of generating Series objects for each row. In contrast, the batch processing approach minimizes this overhead by working directly with NumPy arrays, which are more efficient both in terms of memory layout and computational performance.

In [9]:
# Assume embeddings_df is indexed by node IDs and contains embeddings
embeddings = embeddings_df.to_numpy()

# Map node IDs to their index in the embeddings array for quick lookup
node_id_to_index = {node_id: index for index, node_id in enumerate(embeddings_df.index)}

# Convert edge list source and target to indices
edge_indices = [(node_id_to_index[row['source']], node_id_to_index[row['target']])
                for _, row in edge_list_df.iterrows()]

# Calculate similarities in batches to manage memory usage
batch_size = 1000  # Adjust based on your memory capacity
similarities = []

for i in range(0, len(edge_indices), batch_size):
    batch_edges = edge_indices[i:i+batch_size]
    emb1 = np.array([embeddings[index_pair[0]] for index_pair in batch_edges])
    emb2 = np.array([embeddings[index_pair[1]] for index_pair in batch_edges])
    
    # Calculate batch similarities
    batch_similarities = cosine_similarity(emb1, emb2).diagonal()
    similarities.extend(batch_similarities)

# Add similarities to the edge list DataFrame
edge_list_df['weight'] = similarities


In [10]:
display(edge_list_df.head(100)) 

,source,target,weight
0,u_2,a_51,0.382745
1,u_2,a_52,0.310352
2,u_2,a_53,0.336436
3,u_2,a_54,0.299376
4,u_2,a_55,0.266599
...,...,...,...
95,u_3,a_146,0.458373
96,u_3,a_147,0.802212
97,u_3,a_148,0.559444
98,u_3,a_149,0.816774


We can now export the new updated edge list with cosine similarities as edge weights.

In [11]:
# Optionally save the updated edge list
edge_list_df.to_csv('M2V_edge_list_with_similarity.csv', index=False)